## ***Uncertainty Quantification***

Prof. Dr. M. Frank

------------------------------------------------------------------------

#### **EXERCISE SHEET 7**

In [ ]:
!pip install ghalton
import numpy as np
import sobol_seq
from pyDOE import *
import matplotlib.pyplot as plt
import math
import ghalton

**EXERCISE 1)**

Consider the code block below. 

(a) The code implements a simple example of gPC approximation. Which one? 

(b) How would you evaluate the quality of the approximation for the probability density function and cumulative density function?

(c) What can you observe for different orders of approximation?

In [ ]:
def legendre(n,X) :
  res = []
  for m in range(n+1):
    res.append(scipy.special.lpmv(m,n,X))
  res =np.array(res)
  if res.ndim==1:
    res=np.expand_dims(res,axis=1)
  return res

def Pn(n,x):
  if n == 0:
      return  1
  elif n==1:
      return  x
  elif n>1:
      return  (2*n-1)/n * x * P(n-1,x) - (n-1)/n * P(n-2,x)

Pn = np.vectorize(Pn)

#Evaluate gPC approximation
def YN(z,N,gamma,alpha):
  y=0
  for k in range(N+1):
    y = y + alpha[k]/gamma[k]*phi(k,z) #Assemble polynomial
  return y

#Inverse distribution for Y
def FYinv(y):
  return scipy.special.erfinv(2*y-1) #Gaussian

#Distribution for Y
def FY(y):
  return 1/2*(1+scipy.special.erf(y)) #Gaussian

def fY(y):
  return 1/np.sqrt(math.pi)*np.exp(-y**2) #Gaussian

#Distribution for Z
def FZ(z):
  return (z+1)/2 #Uniform/Legendre

#Density for Z
def fZ(z):
  return 1/2*(z>=-1 and z<=1) #Uniform/Legendre

#Compute orthogonal polynomials
def phi(n,x):
  P = legendre(n,x)
  return P[0,:]

#Averaging
def avg(x):
  return 0.5*(x[1:]+x[0:np.size(x)-1])

fZ = np.vectorize(fZ)

#Integration tolerance
QuadTol = 1e-10

#Order of approximation
N=9

#Domain of support and plot parameters
#gPC reconstruction
z0 = -1
z1 = 1
nz = 100
z = np.linspace(z0,z1,nz)
#pdf of gPC reconstruction
y0 = -2 
y1 = 2
ny = 100
y = np.linspace(y0,y1,ny)
dy = (y1-y0)/ny

gamma = np.empty((N+1,))
alpha = np.empty((N+1,))

#Compute coefficients
for k in range(N+1):
  def f1(z):
    return np.multiply(phi(k,z)**2,fZ(z))
  def f2(z):
    return np.multiply(FYinv(FZ(z)),np.multiply(phi(k,z),fZ(z)))
  gamma[k],_ = integrate.quad_vec(f1,z0,z1,epsabs=QuadTol)
  alpha[k],_ = integrate.quad_vec(f2,z0,z1,epsabs=QuadTol)

#Plot gPC reconstruction
fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(16,8))
ax[0].plot(z,YN(z,N,gamma,alpha))
ax[0].set_xlabel('Z')
ax[0].set_ylabel('Y')
ax[0].set_title('Y_N as function of Z')

#Determine pdf of gPC reconstruction
FYN = np.empty((ny,))
for i in range(np.size(y)):
  def f3(z):
   return np.multiply((YN(z,N,gamma,alpha)<=y[i]),fZ(z))
  FYN[i],_ = integrate.quad_vec(f3,z0,z1,epsabs=QuadTol)

ax[1].plot(y,FYN,label='cdf of Y_N')
ax[1].set_xlabel('y')
ax[1].set_ylabel('FYN(y)')
ax[1].plot(y,FY(y),linestyle='dotted',label='cdf of Y')
ax[1].plot(avg(y),np.diff(FYN)/dy,color='red',label='pdf of Y_N')
ax[1].plot(avg(y),fY(avg(y)),linestyle='dotted',color='red',label='pdf of Y')
_=ax[1].legend(shadow=True, fancybox=True)

In [ ]:
&nbsp;

**EXERCISE 2)**

In this exercise, we prove the properties of Legendre polynomials that were used in the lecture. 
 
 We define the Legendre polynomial of degree $n$ by 
 $$ P_n(x) = \frac{1}{2^n n!} \frac{d^n}{dx^n}(x^2 -1)^n \;.$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (a) Show that $P_n$ is indeed a polynomial of degree $n$.
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (b) Show that the leading coefficient (i.e. the coefficient in front of $x^n$) is 
 $$\frac{(2n)!}{2^n (n!)^2} \;.$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (c) Show that the Legendre polynomials are orthogonal with respect to the standard inner product
 $$(f,g) = \int_{-1}^1 f(x)g(x) dx \;,$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     more specifically that
 $$(P_l,P_m) = \frac{2}{2l+1} \delta_{lm} \;.$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     *Hint:* Show that for any sufficiently smooth function 
 $$2^n n! (f,P_n) = (-1)^n \int_{-1}^{1} f^{(n)}(x)(x^2-1)^n dx $$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     and use 
 $$ \int_0^1 (1-y)^n y^{-1/2} dy = \frac{2^{n+1}n!}{1 \cdot 3 \cdot \cdot \cdot (2n+1)} \;.$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (d) Show that the $P_n$ are eigenfunctions of a different operator in the following sense
 $$\frac{d}{dx} \left((1-x^2)\frac{d}{dx}P_n(x)\right) = -n (n+1) P_n(x) \;.$$
 
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;     *Hint:* Denote the left hand side by $g$. Show that $g+n(n+1)P_n$ is a polynomial of degree $n-1$, express it as a linear combination of $P_0,...,P_{n-1}$ and use orthogonality. 
